In [1]:
!pip install -U essentia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 49.7 MB/s eta 0:00:00


In [2]:
# Setting up the google drive access

# imports to be able to access google drive
from google.colab import drive

# Mount Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Data management
import os

# Math, stats, and data structures
import numpy as np
from scipy.ndimage import filters
#from pylab import plot, show, figure, imshow
import matplotlib.pyplot as plt
import pandas as pd
import json

# Audio processing
import soundfile as sf
import librosa
import essentia.standard as es

In [4]:
song_name = 'Apparat_Goodbye'
#song_name = 'DaftPunk_Crescendolls'

In [5]:
# Import an audio track
# Sample rate 44100
# If stereo, converted to mono

os.chdir("/content/drive/MyDrive/BSL-music")

audio_file_name = f'{song_name}.mp3'
sample_rate = 44100

audio = es.MonoLoader(filename=audio_file_name, sampleRate=sample_rate)()


In [6]:
features, features_frames = es.MusicExtractor(lowlevelFrameSize = 2940, # to get a 30 fps sample rate
                                              lowlevelHopSize = 1470, # to get a 30 fps sample rate
                                              tonalFrameSize = 2940, # to get a 30 fps sample rate
                                              tonalHopSize = 1470 # to get a 30 fps sample rate
                                              #lowlevelFrameSize = 2048, # default
                                              #lowlevelHopSize = 1024, # default
                                              #tonalFrameSize = 4096, # default
                                              #tonalHopSize = 2048 # default
                                              )(audio_file_name)

In [7]:
movement_frame_rate = 30

In [16]:
x_beats = features_frames['rhythm.beats_position']

In [8]:
json_file_name_1 = f'{song_name}_P1_features.json'
json_file_name_2 = f'{song_name}_P2_features.json'
json_file_name_3 = f'{song_name}_P3_features.json'

In [9]:
with open(json_file_name_1) as json_file:
    movement_data_P1 = json.load(json_file)
with open(json_file_name_2) as json_file:
    movement_data_P2 = json.load(json_file)
with open(json_file_name_3) as json_file:
    movement_data_P3 = json.load(json_file)


In [10]:
def P1_P2_P3_to_array(movement_P1, movement_P2, movement_P3):
  # check length (some differ of 1 frame)
  length = len(movement_P1)
  if len(movement_P2) < length:
    length = len(movement_P2)
  if len(movement_P3) < length:
    length = len(movement_P3)
  
  # put P1, P2, and P3 in an array
  movement_total = np.array((movement_P1[:length],
          movement_P2[:length],
          movement_P3[:length]
          ))
  
  return movement_total

In [11]:
def import_feature_P1_P2_P3(path_1, path_2, axis = None,
                            movement_data_P1=movement_data_P1,
                            movement_data_P2=movement_data_P2,
                            movement_data_P3=movement_data_P3):

  if axis == None:
    feature_P1 = []
    for keypoint in movement_data_P1['data']:
      feature_P1.append(keypoint[path_1][path_2])
    feature_P2 = []
    for keypoint in movement_data_P2['data']:
      feature_P2.append(keypoint[path_1][path_2])
    feature_P3 = []
    for keypoint in movement_data_P3['data']:
      feature_P3.append(keypoint[path_1][path_2])

  else:
    feature_P1 = []
    for keypoint in movement_data_P1['data']:
      feature_P1.append(keypoint[path_1][path_2][axis])
    feature_P2 = []
    for keypoint in movement_data_P2['data']:
      feature_P2.append(keypoint[path_1][path_2][axis])
    feature_P3 = []
    for keypoint in movement_data_P3['data']:
      feature_P3.append(keypoint[path_1][path_2][axis])

  # Create one array of total quantity of movement for each piece

  feature_array = P1_P2_P3_to_array(feature_P1, feature_P2,
                                    feature_P3)
  
  return feature_array

In [13]:
def heat_maps(movement_to_plot_1_x, movement_to_plot_1_y,  movement_label_1,
              movement_to_plot_2_x, movement_to_plot_2_y,  movement_label_2,
              song_name = song_name, save=False):
  
  fig,ax = plt.subplots(figsize=(20, 10))

  ax.scatter(movement_to_plot_1_x, movement_to_plot_1_y, c='blue', alpha=0.2, label=movement_label_1+" "+str(song_name))
  ax.scatter(movement_to_plot_2_x, movement_to_plot_2_y, c='yellow', alpha=0.2, label=movement_label_2+" "+str(song_name))

  ax.set_xlim(0, 1920)
  ax.set_ylim(0, 1080)

  ax.set_xlabel("Pixels",
                  fontsize=14)
  ax.set_ylabel("Pixels",
                fontsize=14)

  plt.legend()
  #plt.savefig(f"{song_name} - {movement_label_1} - {movement_label_2} - heat map")
  plt.plot()

  if save==True:
    if not os.path.isdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" heat maps]"}'):
      os.mkdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" heat maps"}')
    os.chdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" heat maps"}')
    plt.savefig(f"{song_name} - {movement_label_1} - {movement_label_2} - heat map")


In [14]:
def window_plot(section_index, section_length, x_feature_to_plot):
  return np.logical_and(section_index*section_length <= x_feature_to_plot,
                        x_feature_to_plot < (section_index+1)*section_length)

In [17]:
def movement_P1_P2_P3(movement_to_plot, movement_label, audio_descriptor_to_plot,
                      audio_descriptor_label, audio_descriptor_y_min,
                      audio_descriptor_y_max, save=False, section_length=60, 
                      smoothing_window=10, x_beats=x_beats):
  
  # We need it both to match the x scales, and to be able to plot sections of a
  # specific length
  coeff = len(movement_to_plot[0])/len(audio_descriptor_to_plot)
  x_audio_descriptor = np.arange(len(audio_descriptor_to_plot)) * coeff
  x_audio_descriptor_sec = x_audio_descriptor /movement_frame_rate
  x_movement_to_plot_sec = np.arange(len(movement_to_plot[0]))/movement_frame_rate

  # Number of sections to plot

  # Number of sections
  section_indexes = np.arange(int(x_audio_descriptor_sec[-1]/section_length)+1)

  for s_i in section_indexes:
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(20, 10))
    # make a plot
    
    x_to_plot = window_plot(s_i, section_length, x_movement_to_plot_sec)
    indices_to_plot = np.where(window_plot(s_i, section_length, x_movement_to_plot_sec))

    ax.plot(x_movement_to_plot_sec[x_to_plot],
            filters.median_filter(movement_to_plot[0][indices_to_plot],
                                  size=smoothing_window),
            label=f'{movement_label}. {song_name}, Participant 1',
            color='blue',
            alpha=1)

    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot[1][indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label}. {song_name}, Participant 2',
          color='red',
          alpha = 1)

    ax.plot(x_movement_to_plot_sec[x_to_plot],
            filters.median_filter(movement_to_plot[2][indices_to_plot],
                                  size=smoothing_window),
            label=f'{movement_label}. {song_name}, Participant 3',
            color='green',
            alpha=1)
    
    ax.vlines(x_beats[window_plot(s_i, section_length, x_beats)],
            np.amin(movement_to_plot),
            np.amax(movement_to_plot),
            colors='violet',
            label='beats')
    

    # twin object for two different y-axis
    ax2=ax.twinx()

    # make a plot with different y-axis using second axis object

    x_descriptor_to_plot = window_plot(s_i, section_length, x_audio_descriptor_sec)
    indices_descriptor_to_plot = np.where(window_plot(s_i, section_length, x_audio_descriptor_sec))

    ax2.plot(x_audio_descriptor_sec[x_descriptor_to_plot],
            audio_descriptor_to_plot[indices_descriptor_to_plot],
            color='orange',
            label=audio_descriptor_label,
            alpha=1)


    # set y lims
    ax.set_ylim(np.amin(movement_to_plot),
            np.amax(movement_to_plot)+(np.amax(movement_to_plot)*0.02))

    ax2.set_ylim(audio_descriptor_y_min,
                audio_descriptor_y_max)

    ax.set_xlabel("Time (sec)",
                  fontsize=14)
    ax.set_ylabel(movement_label,
                  color="blue",
                  fontsize=14)

    ax2.set_ylabel(audio_descriptor_label,
                  color="orange",
                  fontsize=14)

    
    handles, labels = ax.get_legend_handles_labels()
    handles2, lables2 = ax2.get_legend_handles_labels()
    handles.extend(handles2)
    plt.legend(handles=handles)

    plt.show()

    if save==True:
      # Folder to save the plots in
      if not os.path.isdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label)+" "+str(audio_descriptor_label)}'):
        os.mkdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label)+" "+str(audio_descriptor_label)}')
      os.chdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label)+" "+str(audio_descriptor_label)}')
      plt.savefig(f"{song_name} - P 1_2_3 - {movement_label} - {audio_descriptor_label} - Section {s_i+1}")


In [19]:
def wrist_shoulders_P1_P2_P3(movement_to_plot_1, movement_label_1,
                             movement_to_plot_2, movement_label_2,
                             movement_to_plot_3, movement_label_3,
                             movement_to_plot_4, movement_label_4,
                             movement_to_plot_5, movement_label_5,
                             movement_to_plot_6, movement_label_6,
                             movement_file_label, audio_descriptor_to_plot,
                             audio_descriptor_label, audio_descriptor_y_min,
                             audio_descriptor_y_max, save=False, section_length=60, 
                             smoothing_window=10, x_beats=x_beats):
  # x values in seconds.
  # We need it both to match the x scales, and to be able to plot sections of a
  # specific length

  coeff = len(movement_to_plot_1)/len(audio_descriptor_to_plot)
  x_audio_descriptor = np.arange(len(audio_descriptor_to_plot)) * coeff
  x_audio_descriptor_sec = x_audio_descriptor /movement_frame_rate
  x_movement_to_plot_sec = np.arange(len(movement_to_plot_1))/movement_frame_rate

  # Number of sections to plot

  # Number of sections
  section_indexes = np.arange(int(x_audio_descriptor_sec[-1]/section_length)+1)

  for s_i in section_indexes:
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(20, 10))
    # make a plot
    
    x_to_plot = window_plot(s_i, section_length, x_movement_to_plot_sec)
    indices_to_plot = np.where(window_plot(s_i, section_length, x_movement_to_plot_sec))
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_1[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_1}. {song_name}',
          color='red',
          alpha = 1)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_2[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_2}. {song_name}',
          color='red',
          alpha = 0.5)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_3[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_3}. {song_name}',
          color='green',
          alpha = 1)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_4[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_4}. {song_name}',
          color='green',
          alpha = 0.5)

    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_5[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_5}. {song_name}',
          color='blue',
          alpha = 1)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_6[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_6}. {song_name}',
          color='blue',
          alpha = 0.5)


    
    
    ax.vlines(x_beats[window_plot(s_i, section_length, x_beats)],
            0,
            1700,
            colors='violet',
            label='beats')
    

    # twin object for two different y-axis
    ax2=ax.twinx()
    # make a plot with different y-axis using second axis object

    x_descriptor_to_plot = window_plot(s_i, section_length, x_audio_descriptor_sec)
    indices_descriptor_to_plot = np.where(window_plot(s_i, section_length, x_audio_descriptor_sec))

    ax2.plot(x_audio_descriptor_sec[x_descriptor_to_plot],
            audio_descriptor_to_plot[indices_descriptor_to_plot],
            color='orange',
            label=audio_descriptor_label,
            alpha=1)

    # set y lims
    ax.set_ylim(0, 1700)

    ax2.set_ylim(audio_descriptor_y_min,
                audio_descriptor_y_max)

    ax.set_xlabel("Time (sec)",
                  fontsize=14)
    ax.set_ylabel(movement_file_label,
                  color="blue",
                  fontsize=14)

    ax2.set_ylabel(audio_descriptor_label,
                  color="orange",
                  fontsize=14)

    
    handles, labels = ax.get_legend_handles_labels()
    handles2, lables2 = ax2.get_legend_handles_labels()
    handles.extend(handles2)
    plt.legend(handles=handles)

    plt.show()

    if save == True:
      # Folder to save the plots in
      if not os.path.isdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+movement_file_label+" "+str(audio_descriptor_label)+" P1 P2 P3"}'):
        os.mkdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+movement_file_label+" "+str(audio_descriptor_label)+" P1 P2 P3"}')
      os.chdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+movement_file_label+" "+str(audio_descriptor_label)+" P1 P2 P3"}')
      plt.savefig(f"{song_name} - {movement_file_label} - {audio_descriptor_label} - P1 P2 P3 - Section {s_i+1}")



In [12]:
distance_total = import_feature_P1_P2_P3('keypoints_delta', 'delta_total')
angle_total = import_feature_P1_P2_P3('keypoints_delta', 'angle_total')
l_wrist_nose_delta = import_feature_P1_P2_P3('keypoints_LWrist_nose_delta', 'l_wrist_nose_delta')
r_wrist_nose_delta = import_feature_P1_P2_P3('keypoints_RWrist_nose_delta', 'r_wrist_nose_delta')
raw_l_wrist_x = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_l_wrist', 0)
raw_l_wrist_y = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_l_wrist', 1)
raw_r_wrist_x = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_r_wrist', 0)
raw_r_wrist_y = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_r_wrist', 1)
raw_l_shoulder_x = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_l_shoulder', 0)
raw_l_shoulder_y = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_l_shoulder', 1)
raw_r_shoulder_x = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_r_shoulder', 0)
raw_r_shoulder_y = import_feature_P1_P2_P3('raw_wrists_shoulders', 'raw_r_shoulder', 1)

In [ ]:
heat_maps(movement_to_plot_1_x = raw_r_shoulder_x[0],
    movement_to_plot_1_y = raw_r_shoulder_y[0],
    movement_label_1 = 'raw_r_shoulder, P1',
    movement_to_plot_2_x = raw_l_shoulder_x[0],
    movement_to_plot_2_y = raw_l_shoulder_y[0],
    movement_label_2 = 'raw_l_shoulder, P1')

heat_maps(movement_to_plot_1_x = raw_r_shoulder_x[1],
    movement_to_plot_1_y = raw_r_shoulder_y[1],
    movement_label_1 = 'raw_r_shoulder, P2',
    movement_to_plot_2_x = raw_l_shoulder_x[1],
    movement_to_plot_2_y = raw_l_shoulder_y[1],
    movement_label_2 = 'raw_l_shoulder, P2'
)

heat_maps(movement_to_plot_1_x = raw_r_shoulder_x[2],
    movement_to_plot_1_y = raw_r_shoulder_y[2],
    movement_label_1 = 'raw_r_shoulder, P3',
    movement_to_plot_2_x = raw_l_shoulder_x[2],
    movement_to_plot_2_y = raw_l_shoulder_y[2],
    movement_label_2 = 'raw_l_shoulder, P3'
)

heat_maps(movement_to_plot_1_x = raw_r_wrist_x[0],
    movement_to_plot_1_y = raw_r_wrist_y[0],
    movement_label_1 = 'raw_r_wrist, P1',
    movement_to_plot_2_x = raw_l_wrist_x[0],
    movement_to_plot_2_y = raw_l_wrist_y[0],
    movement_label_2 = 'raw_l_wrist, P1'
)

heat_maps(movement_to_plot_1_x = raw_r_wrist_x[1],
    movement_to_plot_1_y = raw_r_wrist_y[1],
    movement_label_1 = 'raw_r_wrist, P2',
    movement_to_plot_2_x = raw_l_wrist_x[1],
    movement_to_plot_2_y = raw_l_wrist_y[1],
    movement_label_2 = 'raw_l_wrist, P2'
)

heat_maps(movement_to_plot_1_x = raw_r_wrist_x[2],
    movement_to_plot_1_y = raw_r_wrist_y[2],
    movement_label_1 = 'raw_r_wrist, P3',
    movement_to_plot_2_x = raw_l_wrist_x[2],
    movement_to_plot_2_y = raw_l_wrist_y[2],
    movement_label_2 = 'raw_l_wrist, P3'
)

In [ ]:
movement_P1_P2_P3(movement_to_plot = distance_total,
    movement_label = 'delta_total',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = angle_total,
    movement_label = 'angle_total',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = distance_total,
    movement_label = 'delta_total',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = angle_total,
    movement_label = 'angle_total',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = l_wrist_nose_delta,
    movement_label = 'l_wrist_nose_delta',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = r_wrist_nose_delta,
    movement_label = 'r_wrist_nose_delta',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = l_wrist_nose_delta,
    movement_label = 'l_wrist_nose_delta',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = r_wrist_nose_delta,
    movement_label = 'r_wrist_nose_delta',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = raw_l_wrist_x,
    movement_label = 'raw_l_wrist_x',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = raw_r_wrist_x,
    movement_label = 'raw_r_wrist_x',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = raw_l_wrist_x,
    movement_label = 'raw_l_wrist_x',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)


movement_P1_P2_P3(movement_to_plot = raw_r_wrist_x,
    movement_label = 'raw_r_wrist_x',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = raw_l_shoulder_x,
    movement_label = 'raw_l_shoulder_x',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = raw_r_shoulder_x,
    movement_label = 'raw_r_shoulder_x',
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

movement_P1_P2_P3(movement_to_plot = raw_l_shoulder_x,
    movement_label = 'raw_l_shoulder_x',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

movement_P1_P2_P3(movement_to_plot = raw_r_shoulder_x,
    movement_label = 'raw_r_shoulder_x',
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

In [ ]:
wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_wrist_x[0],
    movement_label_1 = 'raw_l_wrist_x_P1',
    movement_to_plot_2 = raw_r_wrist_x[0],
    movement_label_2 = 'raw_r_wrist_x_P1',
    movement_to_plot_3 = raw_l_wrist_x[1],
    movement_label_3 = 'raw_l_wrist_x_P2',
    movement_to_plot_4 = raw_r_wrist_x[1],
    movement_label_4 = 'raw_r_wrist_x_P2',
    movement_to_plot_5 = raw_l_wrist_x[2],
    movement_label_5 = 'raw_l_wrist_x_P3',
    movement_to_plot_6 = raw_r_wrist_x[2],
    movement_label_6 = 'raw_r_wrist_x_P3',
    movement_file_label = "Left and right wrist, x axis",
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_wrist_y[0],
    movement_label_1 = 'raw_l_wrist_y_P1',
    movement_to_plot_2 = raw_r_wrist_y[0],
    movement_label_2 = 'raw_r_wrist_y_P1',
    movement_to_plot_3 = raw_l_wrist_y[1],
    movement_label_3 = 'raw_l_wrist_y_P2',
    movement_to_plot_4 = raw_r_wrist_y[1],
    movement_label_4 = 'raw_r_wrist_y_P2',
    movement_to_plot_5 = raw_l_wrist_y[2],
    movement_label_5 = 'raw_l_wrist_y_P3',
    movement_to_plot_6 = raw_r_wrist_y[2],
    movement_label_6 = 'raw_r_wrist_y_P3',
    movement_file_label = "Left and right wrist, y axis",
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_wrist_x[0],
    movement_label_1 = 'raw_l_wrist_x_P1',
    movement_to_plot_2 = raw_r_wrist_x[0],
    movement_label_2 = 'raw_r_wrist_x_P1',
    movement_to_plot_3 = raw_l_wrist_x[1],
    movement_label_3 = 'raw_l_wrist_x_P2',
    movement_to_plot_4 = raw_r_wrist_x[1],
    movement_label_4 = 'raw_r_wrist_x_P2',
    movement_to_plot_5 = raw_l_wrist_x[2],
    movement_label_5 = 'raw_l_wrist_x_P3',
    movement_to_plot_6 = raw_r_wrist_x[2],
    movement_label_6 = 'raw_r_wrist_x_P3',
    movement_file_label = "Left and right wrist, x axis",
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_wrist_y[0],
    movement_label_1 = 'raw_l_wrist_y_P1',
    movement_to_plot_2 = raw_r_wrist_y[0],
    movement_label_2 = 'raw_r_wrist_y_P1',
    movement_to_plot_3 = raw_l_wrist_y[1],
    movement_label_3 = 'raw_l_wrist_y_P2',
    movement_to_plot_4 = raw_r_wrist_y[1],
    movement_label_4 = 'raw_r_wrist_y_P2',
    movement_to_plot_5 = raw_l_wrist_y[2],
    movement_label_5 = 'raw_l_wrist_y_P3',
    movement_to_plot_6 = raw_r_wrist_y[2],
    movement_label_6 = 'raw_r_wrist_y_P3',
    movement_file_label = "Left and right wrist, y axis",
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_shoulder_x[0],
    movement_label_1 = 'raw_l_shoulder_x_P1',
    movement_to_plot_2 = raw_r_shoulder_x[0],
    movement_label_2 = 'raw_r_shoulder_x_P1',
    movement_to_plot_3 = raw_l_shoulder_x[1],
    movement_label_3 = 'raw_l_shoulder_x_P2',
    movement_to_plot_4 = raw_r_shoulder_x[1],
    movement_label_4 = 'raw_r_shoulder_x_P2',
    movement_to_plot_5 = raw_l_shoulder_x[2],
    movement_label_5 = 'raw_l_shoulder_x_P3',
    movement_to_plot_6 = raw_r_shoulder_x[2],
    movement_label_6 = 'raw_r_shoulder_x_P3',
    movement_file_label = "Left and right shoulder, x axis",
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_shoulder_y[0],
    movement_label_1 = 'raw_l_shoulder_y_P1',
    movement_to_plot_2 = raw_r_shoulder_y[0],
    movement_label_2 = 'raw_r_shoulder_y_P1',
    movement_to_plot_3 = raw_l_shoulder_y[1],
    movement_label_3 = 'raw_l_shoulder_y_P2',
    movement_to_plot_4 = raw_r_shoulder_y[1],
    movement_label_4 = 'raw_r_shoulder_y_P2',
    movement_to_plot_5 = raw_l_shoulder_y[2],
    movement_label_5 = 'raw_l_shoulder_y_P3',
    movement_to_plot_6 = raw_r_shoulder_y[2],
    movement_label_6 = 'raw_r_shoulder_y_P3',
    movement_file_label = "Left and right shoulder, y axis",
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary'],
    audio_descriptor_label = 'loudness',
    audio_descriptor_y_min = -70,
    audio_descriptor_y_max = 5
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_shoulder_x[0],
    movement_label_1 = 'raw_l_shoulder_x_P1',
    movement_to_plot_2 = raw_r_shoulder_x[0],
    movement_label_2 = 'raw_r_shoulder_x_P1',
    movement_to_plot_3 = raw_l_shoulder_x[1],
    movement_label_3 = 'raw_l_shoulder_x_P2',
    movement_to_plot_4 = raw_r_shoulder_x[1],
    movement_label_4 = 'raw_r_shoulder_x_P2',
    movement_to_plot_5 = raw_l_shoulder_x[2],
    movement_label_5 = 'raw_l_shoulder_x_P3',
    movement_to_plot_6 = raw_r_shoulder_x[2],
    movement_label_6 = 'raw_r_shoulder_x_P3',
    movement_file_label = "Left and right shoulder, x axis",
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

wrist_shoulders_P1_P2_P3(movement_to_plot_1 = raw_l_shoulder_y[0],
    movement_label_1 = 'raw_l_shoulder_y_P1',
    movement_to_plot_2 = raw_r_shoulder_y[0],
    movement_label_2 = 'raw_r_shoulder_y_P1',
    movement_to_plot_3 = raw_l_shoulder_y[1],
    movement_label_3 = 'raw_l_shoulder_y_P2',
    movement_to_plot_4 = raw_r_shoulder_y[1],
    movement_label_4 = 'raw_r_shoulder_y_P2',
    movement_to_plot_5 = raw_l_shoulder_y[2],
    movement_label_5 = 'raw_l_shoulder_y_P3',
    movement_to_plot_6 = raw_r_shoulder_y[2],
    movement_label_6 = 'raw_r_shoulder_y_P3',
    movement_file_label = "Left and right shoulder, y axis",
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10),
    audio_descriptor_label = 'spectral centroid',
    audio_descriptor_y_min = 0,
    audio_descriptor_y_max = 3000
)

In [ ]:
# Wrists and shoulders pairs

# Length of the moving average to smooth movemnt features
smoothing_window = 10

# Let's put the beats in a variable
x_beats = features_frames['rhythm.beats_position']

section_length = 60

for i in range(7, 10):
  
  set_to_plot = i

  if set_to_plot == 1:
    movement_to_plot_1 = raw_l_wrist_x[0]
    movement_label_1 = 'left_wrist_P1'
    movement_to_plot_2 = raw_r_wrist_x[0]
    movement_label_2 = 'right_wrist_P1'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5
  elif set_to_plot == 2:
    movement_to_plot_1 = raw_l_wrist_x[1]
    movement_label_1 = 'left_wrist_P2'
    movement_to_plot_2 = raw_r_wrist_x[1]
    movement_label_2 = 'right_wrist_P2'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5
  elif set_to_plot == 3:
    movement_to_plot_1 = raw_l_wrist_x[2]
    movement_label_1 = 'left_wrist_P3'
    movement_to_plot_2 = raw_r_wrist_x[2]
    movement_label_2 = 'right_wrist_P3'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5

  elif set_to_plot == 4:
    movement_to_plot_1 = raw_l_shoulder_x[0]
    movement_label_1 = 'left_shoulder_P1'
    movement_to_plot_2 = raw_r_shoulder_x[0]
    movement_label_2 = 'right_shoulder_P1'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5
  elif set_to_plot == 5:
    movement_to_plot_1 = raw_l_shoulder_x[1]
    movement_label_1 = 'left_shoulder_P2'
    movement_to_plot_2 = raw_r_shoulder_x[1]
    movement_label_2 = 'right_shoulder_P2'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5
  elif set_to_plot == 6:
    movement_to_plot_1 = raw_l_shoulder_x[2]
    movement_label_1 = 'left_shoulder_P3'
    movement_to_plot_2 = raw_r_shoulder_x[2]
    movement_label_2 = 'right_shoulder_P3'
    audio_descriptor_to_plot = features_frames['lowlevel.loudness_ebu128.momentary']
    audio_descriptor_label = 'loudness'
    audio_descriptor_y_min = -70
    audio_descriptor_y_max = 5
  
  elif set_to_plot == 7:
    movement_to_plot_1 = raw_l_wrist_x[0]
    movement_label_1 = 'left_wrist_P1'
    movement_to_plot_2 = raw_r_wrist_x[0]
    movement_label_2 = 'right_wrist_P1'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000
  elif set_to_plot == 8:
    movement_to_plot_1 = raw_l_wrist_x[1]
    movement_label_1 = 'left_wrist_P2'
    movement_to_plot_2 = raw_r_wrist_x[1]
    movement_label_2 = 'right_wrist_P2'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000
  elif set_to_plot == 9:
    movement_to_plot_1 = raw_l_wrist_x[2]
    movement_label_1 = 'left_wrist_P3'
    movement_to_plot_2 = raw_r_wrist_x[2]
    movement_label_2 = 'right_wrist_P3'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000

  elif set_to_plot == 10:
    movement_to_plot_1 = raw_l_shoulder_x[0]
    movement_label_1 = 'left_shoulder_P1'
    movement_to_plot_2 = raw_r_shoulder_x[0]
    movement_label_2 = 'right_shoulder_P1'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000
  elif set_to_plot == 11:
    movement_to_plot_1 = raw_l_shoulder_x[1]
    movement_label_1 = 'left_shoulder_P2'
    movement_to_plot_2 = raw_r_shoulder_x[1]
    movement_label_2 = 'right_shoulder_P2'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000
  elif set_to_plot == 12:
    movement_to_plot_1 = raw_l_shoulder_x[2]
    movement_label_1 = 'left_shoulder_P3'
    movement_to_plot_2 = raw_r_shoulder_x[2]
    movement_label_2 = 'right_shoulder_P3'
    audio_descriptor_to_plot = filters.median_filter(features_frames['lowlevel.spectral_centroid'], size=10)
    audio_descriptor_label = 'spectral centroid'
    audio_descriptor_y_min = 0
    audio_descriptor_y_max = 3000


  # x values in seconds.
  # We need it both to match the x scales, and to be able to plot sections of a
  # specific length

  coeff = len(movement_to_plot_1)/len(audio_descriptor_to_plot)
  x_audio_descriptor = np.arange(len(audio_descriptor_to_plot)) * coeff
  x_audio_descriptor_sec = x_audio_descriptor /movement_frame_rate
  x_movement_to_plot_sec = np.arange(len(movement_to_plot_1))/movement_frame_rate

  # Number of sections to plot

  # Number of sections
  section_indexes = np.arange(int(x_audio_descriptor_sec[-1]/section_length)+1)

  # Folder to save the plots in
  if not os.path.isdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label_1)+" "+str(movement_label_2)+" "+str(audio_descriptor_label)+"P1 P2 P3"}'):
    os.mkdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label_1)+" "+str(movement_label_2)+" "+str(audio_descriptor_label)+"P1 P2 P3"}')

  os.chdir(f'/content/drive/MyDrive/BSL-music/Plots/{song_name+" "+str(movement_label_1)+" "+str(movement_label_2)+" "+str(audio_descriptor_label)+"P1 P2 P3"}')

  for s_i in section_indexes:
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(20, 10))
    # make a plot
    
    x_to_plot = window_plot(s_i, section_length, x_movement_to_plot_sec)
    indices_to_plot = np.where(window_plot(s_i, section_length, x_movement_to_plot_sec))
    
#    ax.plot(x_movement_to_plot_sec[x_to_plot],
#            filters.median_filter(movement_to_plot_1[indices_to_plot],
#                                  size=smoothing_window),
#            label=f'{movement_label_1}. {song_name}',
#            color='blue',
#            alpha=1)

    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(movement_to_plot_2[indices_to_plot],
                                  size=smoothing_window),
          label=f'{movement_label_2}. {song_name}',
          color='red',
          alpha = 1)


    label_P1_P2_P3 = "P1 P2 P3 left"

    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(raw_l_wrist_x[0][indices_to_plot],
                                  size=smoothing_window),
          label=f'left_wrist P1. {song_name}',
          alpha = 1)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(raw_l_wrist_x[1][indices_to_plot],
                                  size=smoothing_window),
          label=f'left_wrist P2. {song_name}',
          alpha = 1)
    
    ax.plot(x_movement_to_plot_sec[x_to_plot],
          filters.median_filter(raw_l_wrist_x[2][indices_to_plot],
                                  size=smoothing_window),
          label=f'left_wrist P3. {song_name}',
          alpha = 1)
    
#    label_P1_P2_P3 = "P1 P2 P3 right"

#    ax.plot(x_movement_to_plot_sec[x_to_plot],
#          filters.median_filter(raw_r_wrist_x[0][indices_to_plot],
#                                  size=smoothing_window),
#          label=f'right_wrist_raw P1. {song_name}',
#          alpha = 1)
#    
#    ax.plot(x_movement_to_plot_sec[x_to_plot],
#          filters.median_filter(raw_r_wrist_x[1][indices_to_plot],
#                                  size=smoothing_window),
#          label=f'right_wrist_raw P2. {song_name}',
#          alpha = 1)
#    
#    ax.plot(x_movement_to_plot_sec[x_to_plot],
#          filters.median_filter(raw_r_wrist_x[2][indices_to_plot],
#                                  size=smoothing_window),
#          label=f'right_wrist_raw P3. {song_name}',
#          alpha = 1)
    

    
    
    ax.vlines(x_beats[window_plot(s_i, section_length, x_beats)],
            np.amin(np.minimum(movement_to_plot_1, movement_to_plot_2)),
            np.amax(np.maximum(movement_to_plot_1, movement_to_plot_2)),
            colors='violet',
            label='beats')
    

    # twin object for two different y-axis
    ax2=ax.twinx()
    # make a plot with different y-axis using second axis object



    x_descriptor_to_plot = window_plot(s_i, section_length, x_audio_descriptor_sec)
    indices_descriptor_to_plot = np.where(window_plot(s_i, section_length, x_audio_descriptor_sec))

    ax2.plot(x_audio_descriptor_sec[x_descriptor_to_plot],
            audio_descriptor_to_plot[indices_descriptor_to_plot],
            color='orange',
            label=audio_descriptor_label,
            alpha=1)

    #ax2.plot(x_audio_descriptor_sec[x_descriptor_to_plot],
    #        filters.median_filter(audio_descriptor_to_plot[indices_descriptor_to_plot],
    #                              size=smoothing_window),
    #        label=audio_descriptor_label,
    #        color='orange',
    #        alpha=1)


    # set y lims
    ax.set_ylim(np.amin(np.minimum(movement_to_plot_1, movement_to_plot_2)),
            np.amax((np.maximum(movement_to_plot_1, movement_to_plot_2)))+
                    (np.amax((np.maximum(movement_to_plot_1, movement_to_plot_2))*0.02)))

    ax2.set_ylim(audio_descriptor_y_min,
                audio_descriptor_y_max)

    ax.set_xlabel("Time (sec)",
                  fontsize=14)
    ax.set_ylabel(movement_label_1+str(" "+movement_label_2),
                  color="blue",
                  fontsize=14)

    ax2.set_ylabel(audio_descriptor_label,
                  color="orange",
                  fontsize=14)

    
    handles, labels = ax.get_legend_handles_labels()
    handles2, lables2 = ax2.get_legend_handles_labels()
    

    handles.extend(handles2)

    plt.legend(handles=handles)
    #ax2.legend()
    #plt.savefig(f"{song_name} - {movement_label_1} - {movement_label_2} - {audio_descriptor_label} - {label_P1_P2_P3} - Section {s_i+1}")

    #plt.show()